In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
%matplotlib inline

In [51]:
df_train = pd.read_csv('../data/training_variants.csv',index_col='ID')
df_test = pd.read_csv('../data/test_variants.csv',index_col='ID')
df = pd.concat((df_train,df_test))
df = df.reset_index(drop=True)
df.head()

,Class,Gene,Variation
0,1.0,FAM58A,Truncating Mutations
1,2.0,CBL,W802*
2,2.0,CBL,Q249E
3,3.0,CBL,N454D
4,4.0,CBL,L399V


In [52]:
df = pd.concat((df_train,df_test))
df['text'] =  pd.concat((pd.read_pickle('../data/genes-train.pk').text,pd.read_pickle('../data/genes-test.pk').text))
df.head()

,Class,Gene,Variation,text
ID,,,,
0,1.0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,2.0,CBL,W802*,Abstract Background Non-small cell lung canc...
2,2.0,CBL,Q249E,Abstract Background Non-small cell lung canc...
3,3.0,CBL,N454D,Recent evidence has demonstrated that acquired...
4,4.0,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...


In [53]:
df['gene_present'] = df.apply(lambda r: r['Gene'] in r['text'].split(' '),axis=1)
df['variant_present'] = df.apply(lambda r: r['Variation'] in r['text'].split(' '),axis=1)
for i in range(2,10):
    df['gene_let_{}'.format(i)] = df['Gene'].map(lambda x: ord(x[i]) if i<len(x) else -1)
for i in range(3,20):
    df['var_let_{}'.format(i)] = df['Variation'].map(lambda x: ord(x[i]) if i<len(x) else -1)

In [54]:
df.head()
y = df.Class
X = df[df.columns.difference(['Gene','Variation','text','Class'])]

In [58]:
X = X[['gene_present','variant_present'] + list(X.columns.difference(['gene_present','variant_present']))]
X.head()

,gene_present,variant_present,gene_let_2,gene_let_3,gene_let_4,gene_let_5,gene_let_6,gene_let_7,gene_let_8,gene_let_9,...,var_let_17,var_let_18,var_let_19,var_let_3,var_let_4,var_let_5,var_let_6,var_let_7,var_let_8,var_let_9
ID,,,,,,,,,,,,,,,,,,,,,
0,True,False,77,53,56,65,-1,-1,-1,-1,...,111,110,115,110,99,97,116,105,110,103
1,True,True,76,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,50,42,-1,-1,-1,-1,-1
2,True,True,76,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,57,69,-1,-1,-1,-1,-1
3,True,True,76,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,52,68,-1,-1,-1,-1,-1
4,True,True,76,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,57,86,-1,-1,-1,-1,-1


In [67]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
pipeline = Pipeline([('ohe',OneHotEncoder(categorical_features=
                        [0 if x in ['gene_present','variant_present'] else 1 for x in X.columns])),
                     ('xgb',XGBClassifier(n_estimators=500,n_jobs=-1,colsample_bytree=.5))])

In [72]:
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
X,y = shuffle(X.values[:df_train.shape[0]],y.values[:df_train.shape[0]])
cross_val_score(pipeline,X,y,scoring='neg_log_loss',cv=5)

array([-1.25203105, -1.14895664, -1.14922708, -1.1436171 , -1.152031  ])

In [69]:
y

ID
0       1.0
1       2.0
2       2.0
3       3.0
4       4.0
5       4.0
6       5.0
7       1.0
8       4.0
9       4.0
10      4.0
11      4.0
12      4.0
13      4.0
14      5.0
15      4.0
16      1.0
17      4.0
18      5.0
19      4.0
20      6.0
21      4.0
22      4.0
23      4.0
24      4.0
25      4.0
26      4.0
27      4.0
28      7.0
29      4.0
       ... 
5638    NaN
5639    NaN
5640    NaN
5641    NaN
5642    NaN
5643    NaN
5644    NaN
5645    NaN
5646    NaN
5647    NaN
5648    NaN
5649    NaN
5650    NaN
5651    NaN
5652    NaN
5653    NaN
5654    NaN
5655    NaN
5656    NaN
5657    NaN
5658    NaN
5659    NaN
5660    NaN
5661    NaN
5662    NaN
5663    NaN
5664    NaN
5665    NaN
5666    NaN
5667    NaN
Name: Class, Length: 8989, dtype: float64

In [70]:
X.shape

(8989, 27)